In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [34]:
sample_submission = pd.read_csv("sample_submission.csv")

# Limpeza dos dados de entrada

In [4]:
sample_tweets_train=train_df['text']
sample_tweets_test=test_df['text']

In [5]:
def limpeza(tweets):
    tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
    tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
    tweets = tweets.str.replace(r'[0-9]','')
    tweets = tweets.replace("\\x89Û_",'')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÒ','')
    tweets = tweets.replace('\x89Û','')
    tweets = tweets.replace('\x89Û÷','')
    tweets = tweets.replace('\x89ûï','')
    #retirada de símbolos não alfanuméticos 
    tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")
    tweets = tweets.str.lower()
    return tweets

In [6]:
sample_tweets_train = limpeza(sample_tweets_train)
sample_tweets_test = limpeza(sample_tweets_test)

C:\Users\yfrom\AppData\Local\Temp\ipykernel_12048\1236119256.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_12048\1236119256.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_12048\1236119256.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'[0-9]','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_12048\1236119256.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")


In [7]:
stop_words = set(stopwords.words("english"))
def lematizar(tweets):
    sample_tweets= []
    for tweet in tweets:
        filtered_list = []
        words_lemmatized = []
        words = []
        words = word_tokenize(tweet) 
        words_lemmatized = [lemmatizer.lemmatize(word) for word in words] 
        for word in words_lemmatized:
            if word not in stop_words:
                filtered_list.append(word)
        sample_tweets.append(filtered_list)
    return sample_tweets 

In [8]:
sample_tweets_train = lematizar(sample_tweets_train)
sample_tweets_test = lematizar(sample_tweets_test)

In [9]:
train_df['words']= sample_tweets_train
test_df['words']= sample_tweets_test
test_df = test_df.drop(columns=['location', 'keyword','id','text'])
train_df = train_df.drop(columns=['location', 'keyword','id','text'])
train_df['text'] = train_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))
test_df['text'] = test_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))

In [14]:
train_df.head()

,target,words,text
0,1,"[deed, reason, earthquake, may, allah, forgive...",deed reason earthquake may allah forgive u
1,1,"[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,1,"[resident, asked, 'shelter, place, ', notified...",resident asked 'shelter place ' notified offic...
3,1,"[people, receive, wildfire, evacuation, order,...",people receive wildfire evacuation order calif...
4,1,"[got, sent, photo, ruby, alaska, smoke, wildfi...",got sent photo ruby alaska smoke wildfire pour...


In [15]:
test_df.head()

,words,text
0,"[happened, terrible, car, crash]",happened terrible car crash
1,"[heard, earthquake, different, city, stay, saf...",heard earthquake different city stay safe ever...
2,"[forest, fire, spot, pond, goose, fleeing, acr...",forest fire spot pond goose fleeing across str...
3,"[apocalypse, lighting, spokane, wildfire]",apocalypse lighting spokane wildfire
4,"[typhoon, soudelor, kill, china, taiwan]",typhoon soudelor kill china taiwan


# tf-idf

In [16]:
tfidf_vectorizer = TfidfVectorizer()
count_treino_tfidf = tfidf_vectorizer.fit_transform(train_df['text'])
count_teste_tfidf = tfidf_vectorizer.transform(test_df['text'])

# Regressão Logística

In [35]:
clf_tfidf = LogisticRegression(C=1, class_weight='dict', solver='newton-cg',multi_class='auto', n_jobs=-1, random_state=40,max_iter= 1000)
clf_tfidf.fit(count_treino_tfidf, train_df['target'])
sample_submission['target'] = clf_tfidf.predict(count_teste_tfidf)

In [37]:
sample_submission.to_csv("submission.csv", index=False)